In [1]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_ollama import ChatOllama
from langchain.tools import tool
import random
import pprint

In [8]:
book_database = {
    "Thinking, Fast and Slow": {"creativity": 2, "accuracy": 5, "knowledge": 5},
    "The Alchemist": {"creativity": 5, "accuracy": 2, "knowledge": 3},
    "Sapiens": {"creativity": 3, "accuracy": 4, "knowledge": 5},
    "1984": {"creativity": 5, "accuracy": 2, "knowledge": 3},
    "Harry Potter": {"creativity": 5, "accuracy": 3, "knowledge": 2},
}

# Initial user state
state = {
    "preferences": {"creativity": 8, "accuracy": 4, "knowledge": 6},
    "recommended_book": "None",
    "feedback": "0.0",
    "weights": {"creativity": 5.0, "accuracy": 5.0, "knowledge": 5.0},
}

In [3]:
from langchain.tools import tool

@tool
def recommend_book() -> str:
    """Recommend the best book based on weighted user preferences."""
    global state
    weights = state["weights"]
    scores = {}
    for book, traits in book_database.items():
        score = sum(weights.get(k, 0) * traits.get(k, 0) for k in traits)
        scores[book] = score
    best_book = max(scores, key=scores.get)
    state["recommended_book"] = best_book
    return f"Recommended book: {best_book} based on weights {weights}"

@tool
def simulate_feedback() -> str:
    """Simulate user feedback based on how well the recommended book aligns with preferences."""
    global state
    book = state["recommended_book"]
    prefs = state["preferences"]
    traits = book_database.get(book, {})
    alignment = sum(
        1 if traits.get(k, 0) >= prefs.get(k, 5) else -1
        for k in prefs
    )
    feedback = "good" if alignment >= 0 else "bad"
    state["feedback"] = feedback
    return f"User gave '{book}' a feedback: {feedback}"

@tool
def update_weights() -> str:
    """Update weights based on feedback to improve future recommendations."""
    global state
    feedback = state["feedback"]
    book = state["recommended_book"]
    traits = book_database[book]
    adjustment = 1 if feedback == "good" else -1

    for k in state["weights"]:
        delta = 0.1 * adjustment * traits.get(k, 0)
        state["weights"][k] += delta
        state["weights"][k] = max(0.0, min(10.0, state["weights"][k]))
    return f"Updated weights: {state['weights']}"

In [ ]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_ollama import ChatOllama

# Load local Ollama LLM
llm = ChatOllama(model="deepseek-r1:1.5b", temperature=0.7)

# Define tools with proper name and description
tools = [
    Tool.from_function(
        func=recommend_book,
        name="RecommendBook",
        description="Recommend a book based on user preferences."
    ),
    Tool.from_function(
        func=simulate_feedback,
        name="SimulateFeedback",
        description="Simulate user feedback for a given recommendation."
    ),
    Tool.from_function(
        func=update_weights,
        name="UpdateWeights",
        description="Update the preference weights based on feedback."
    ),
]

# Create the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
# Run 3 rounds of book recommendation using the agent
for i in range(3):
    print(f"\n=== Round {i+1} ===")

    # Step 1: Recommend a book
    prompt = (
        f"User preferences: {state['preferences']}.\n"
        f"Available books: {', '.join(book_database.keys())}.\n"
        f"Use the RecommendBook tool to select a book based on user preferences and current weights.\n"
        f"Format: Action: RecommendBook\nAction Input: Select a book\nFinal Answer: [Book title from tool]"
    )
    try:
        response = agent.run(prompt)
        matched_title = next(
            (title for title in book_database if title.lower() == response.lower()),
            None
        ) if response else None
    except Exception as e:
        matched_title = None
        print(f"[Error] Agent failed: {str(e)}")
        continue

    if not matched_title:
        print("[Error] Invalid book title returned by agent:", response)
        continue

    state["recommended_book"] = matched_title

    # Step 2: Generate explanation
    traits = book_database[matched_title]
    explain_prompt = (
        f"User preferences: {state['preferences']}. Book traits: {traits}.\n"
        f"Why is '{matched_title}' a good match? Reply with ONE short sentence.\n"
        f"Do NOT use tools; provide the answer directly.\n"
        f"Format: Final Answer: [One-sentence explanation]"
    )
    explanation = agent.run(explain_prompt).replace("Final Answer: ", "").strip()

    # Step 3: Simulate feedback
    feedback_prompt = (
        f"Use the SimulateFeedback tool to evaluate the recommended book: {matched_title}.\n"
        f"Format: Action: SimulateFeedback\nAction Input: Evaluate {matched_title}\nFinal Answer: [Feedback from tool]"
    )
    feedback = agent.run(feedback_prompt).strip()

    # Step 4: Update weights
    update_prompt = (
        f"Use the UpdateWeights tool to adjust weights based on feedback for {matched_title}.\n"
        f"Format: Action: UpdateWeights\nAction Input: Adjust weights for {matched_title}\nFinal Answer: [Weights from tool]"
    )
    weights = agent.run(update_prompt).strip()

    # Step 5: Print concise result
    print(f"[Recommendation] {matched_title} | [Reason] {explanation} | [Feedback] {feedback} | [Weights] {weights}")